In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import torch
from torch_geometric import nn
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [38]:
test_edges['user_id'].value_counts()

user_id
182401    725
75304     668
151994    626
116571    605
152144    603
         ... 
103374      1
86475       1
32121       1
117096      1
202680      1
Name: count, Length: 101696, dtype: int64

In [66]:
user_feat = train_features.groupby("user_id").agg({
    "order_number":"nunique",
    "order_dow":["min", "max", "mean", "count", "std"],
    "order_hour_of_day":["min", "max", "mean", "count", "std"],
    "days_since_prior_order":["min", "max", "mean", "count", "std"],
    "reordered":"mean",
})

In [67]:
user_feat

order_number order_dow                                 
             nunique       min max      mean count       std   
user_id                                                        
1                  7         1   4  2.842105    38  1.053343  \
2                 11         1   5  1.907285   151  0.859077   
3                  9         0   3  0.915493    71  1.227616   
7                 17         0   6  1.732955   176  1.907053   
13                 9         0   6  2.660714    56  2.931191   
...              ...       ...  ..       ...   ...       ...   
206202            19         0   5  2.209581   167  1.520274   
206206            64         0   6  2.309963   271  1.958801   
206207            13         0   6  2.611111   180  2.083156   
206208            46         0   6  2.821940   629  1.763057   
206209            10         0   5  3.065217    92  1.458619   

        order_hour_of_day                                  
                      min max       mean count       std   
user_id                                                    
1                       7  15   9.736842    38  3.193610  \
2                       9  15  10.635762   151  1.783194   
3                      14  19  16.380282    71  1.505857   
7                       7  20  14.335227   176  3.915699   
13                      8  21  14.500000    56  4.484316   
...                   ...  ..        ...   ...       ...   
206202                  0  23  15.029940   167  6.848116   
206206                 12  20  17.007380   271  2.090530   
206207                  4  21  14.005556   180  4.511311   
206208                  8  23  14.154213   629  3.167822   
206209                 10  18  13.467391    92  3.036232   

        days_since_prior_order                                   reordered  
                           min   max       mean count        std      mean  
user_id                                                                     
1                         15.0  29.0  22.424242    33   5.250180  0.657895  
2                          3.0  30.0  13.739130   138   8.416446  0.476821  
3                          7.0  21.0  10.639344    61   5.039950  0.549296  
7                          2.0  30.0  14.865854   164  11.036206  0.659091  
13                         6.0  13.0   7.568627    51   2.012510  0.589286  
...                        ...   ...        ...   ...        ...       ...  
206202                     3.0  24.0  11.826087   161   5.233982  0.640719  
206206                     0.0  15.0   3.674157   267   3.112340  0.468635  
206207                     1.0  30.0  16.288462   156  12.049901  0.527778  
206208                     0.0  20.0   7.427877   617   4.155469  0.691574  
206209                     3.0  30.0  21.379747    79  10.073199  0.402174  

[101696 rows x 17 columns]

In [68]:
train_features.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_count,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,10,196,1,0
0,2539329,1,1,2,8,NaN,10,14084,2,0
0,2539329,1,1,2,8,NaN,10,12427,3,0
0,2539329,1,1,2,8,NaN,10,26088,4,0
0,2539329,1,1,2,8,NaN,10,26405,5,0


In [69]:
user_feat_1 = train_features.groupby(["user_id", "order_number"])["add_to_cart_order"].max().reset_index().rename(columns={"add_to_cart_order":"order_size"})
user_feat_1 = user_feat_1.groupby(["user_id"]).agg({"order_size":["min", "max", "mean", "count", "std"]})
user_feat_1

order_size                               
               min max       mean count       std
user_id                                          
1                4   8   5.428571     7  1.272418
2                5  26  13.727273    11  6.084257
3                5  11   7.888889     9  2.147350
7                1  24  10.352941    17  6.234392
13               4  10   6.222222     9  2.048034
...            ...  ..        ...   ...       ...
206202           2  16   8.789474    19  4.315076
206206           1  24   4.234375    64  3.264733
206207           5  25  13.846154    13  6.094344
206208           4  28  13.673913    46  5.521893
206209           2  16   9.200000    10  5.072803

[101696 rows x 5 columns]

In [70]:
user_feat.columns = ["_".join(x) for x in user_feat.columns]
user_feat_1.columns = ["_".join(x) for x in user_feat_1.columns]

In [71]:
user_feat_comb = user_feat.join(user_feat_1)
user_feat_comb

,order_number_nunique,order_dow_min,order_dow_max,order_dow_mean,order_dow_count,order_dow_std,order_hour_of_day_min,order_hour_of_day_max,order_hour_of_day_mean,order_hour_of_day_count,...,days_since_prior_order_max,days_since_prior_order_mean,days_since_prior_order_count,days_since_prior_order_std,reordered_mean,order_size_min,order_size_max,order_size_mean,order_size_count,order_size_std
user_id,,,,,,,,,,,,,,,,,,,,,
1,7,1,4,2.842105,38,1.053343,7,15,9.736842,38,...,29.0,22.424242,33,5.250180,0.657895,4,8,5.428571,7,1.272418
2,11,1,5,1.907285,151,0.859077,9,15,10.635762,151,...,30.0,13.739130,138,8.416446,0.476821,5,26,13.727273,11,6.084257
3,9,0,3,0.915493,71,1.227616,14,19,16.380282,71,...,21.0,10.639344,61,5.039950,0.549296,5,11,7.888889,9,2.147350
7,17,0,6,1.732955,176,1.907053,7,20,14.335227,176,...,30.0,14.865854,164,11.036206,0.659091,1,24,10.352941,17,6.234392
13,9,0,6,2.660714,56,2.931191,8,21,14.500000,56,...,13.0,7.568627,51,2.012510,0.589286,4,10,6.222222,9,2.048034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206202,19,0,5,2.209581,167,1.520274,0,23,15.029940,167,...,24.0,11.826087,161,5.233982,0.640719,2,16,8.789474,19,4.315076
206206,64,0,6,2.309963,271,1.958801,12,20,17.007380,271,...,15.0,3.674157,267,3.112340,0.468635,1,24,4.234375,64,3.264733
206207,13,0,6,2.611111,180,2.083156,4,21,14.005556,180,...,30.0,16.288462,156,12.049901,0.527778,5,25,13.846154,13,6.094344


In [76]:
user_feat__sum = user_feat_comb.describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99]).T
user_feat__sum

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
order_number_nunique,101696.0,23.116298,18.436710,7.000000,10.000000,16.000000,29.000000,48.000000,62.000000,96.000000,96.000000
order_dow_min,101696.0,0.221808,0.527373,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,6.000000
order_dow_max,101696.0,5.636161,0.741874,0.000000,5.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
order_dow_mean,101696.0,2.763063,0.838596,0.000000,2.245455,2.770833,3.279318,3.805387,4.152778,4.846866,6.000000
order_dow_count,101696.0,232.893772,235.469985,7.000000,83.000000,153.000000,295.000000,528.000000,712.000000,1140.000000,3618.000000
order_dow_std,101696.0,1.844767,0.471252,0.000000,1.554150,1.888796,2.161584,2.417540,2.570930,2.804114,3.113247
order_hour_of_day_min,101696.0,7.013304,3.387211,0.000000,6.000000,8.000000,9.000000,10.000000,12.000000,14.000000,21.000000
order_hour_of_day_max,101696.0,19.765222,2.581493,6.000000,18.000000,20.000000,22.000000,23.000000,23.000000,23.000000,23.000000
order_hour_of_day_mean,101696.0,13.447464,1.922703,1.293333,12.166667,13.441860,14.709432,15.896552,16.613596,17.997654,22.364073
order_hour_of_day_count,101696.0,232.893772,235.469985,7.000000,83.000000,153.000000,295.000000,528.000000,712.000000,1140.000000,3618.000000


In [75]:
count_cols = [col for col in user_feat_comb.columns if "count" in col]
count_cols

['order_dow_count',
 'order_hour_of_day_count',
 'days_since_prior_order_count',
 'order_size_count']

In [79]:
user_feat__sum.loc['order_dow_count', "50%"]

153.0

In [80]:
# clip
for col in count_cols:
    user_feat_comb[col] = user_feat_comb[col].apply(lambda x: x if x <= user_feat__sum.loc[col, "50%"] * 3 else user_feat__sum.loc[col, "50%"] * 3)

In [81]:
user_feat_comb.describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99]).T

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
order_number_nunique,101696.0,23.116298,18.436710,7.000000,10.000000,16.000000,29.000000,48.000000,62.000000,96.000000,96.000000
order_dow_min,101696.0,0.221808,0.527373,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,6.000000
order_dow_max,101696.0,5.636161,0.741874,0.000000,5.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
order_dow_mean,101696.0,2.763063,0.838596,0.000000,2.245455,2.770833,3.279318,3.805387,4.152778,4.846866,6.000000
order_dow_count,101696.0,198.446537,142.086943,7.000000,83.000000,153.000000,295.000000,459.000000,459.000000,459.000000,459.000000
order_dow_std,101696.0,1.844767,0.471252,0.000000,1.554150,1.888796,2.161584,2.417540,2.570930,2.804114,3.113247
order_hour_of_day_min,101696.0,7.013304,3.387211,0.000000,6.000000,8.000000,9.000000,10.000000,12.000000,14.000000,21.000000
order_hour_of_day_max,101696.0,19.765222,2.581493,6.000000,18.000000,20.000000,22.000000,23.000000,23.000000,23.000000,23.000000
order_hour_of_day_mean,101696.0,13.447464,1.922703,1.293333,12.166667,13.441860,14.709432,15.896552,16.613596,17.997654,22.364073
order_hour_of_day_count,101696.0,198.446537,142.086943,7.000000,83.000000,153.000000,295.000000,459.000000,459.000000,459.000000,459.000000


In [82]:
user_feat_comb_norm = (user_feat_comb - user_feat_comb.mean()) / user_feat_comb.std()
user_feat_comb_norm

,order_number_nunique,order_dow_min,order_dow_max,order_dow_mean,order_dow_count,order_dow_std,order_hour_of_day_min,order_hour_of_day_max,order_hour_of_day_mean,order_hour_of_day_count,...,days_since_prior_order_max,days_since_prior_order_mean,days_since_prior_order_count,days_since_prior_order_std,reordered_mean,order_size_min,order_size_max,order_size_mean,order_size_count,order_size_std
user_id,,,,,,,,,,,,,,,,,,,,,
1,-0.874142,1.47560,-2.205443,0.094255,-1.129214,-1.679407,-0.003928,-1.845917,-1.929898,-1.129214,...,0.484797,1.961316,-1.133404,-0.549137,0.651085,0.156213,-1.113417,-0.820971,-1.065751,-1.287883
2,-0.657183,1.47560,-0.857505,-1.020489,-0.333926,-2.091640,0.586528,-1.845917,-1.462369,-0.333926,...,0.637256,0.332954,-0.354093,0.660219,-0.376988,0.470001,0.615917,0.646668,-0.765383,0.655785
3,-0.765662,-0.42059,-3.553381,-2.203171,-0.896962,-1.309597,2.062669,-0.296426,1.525362,-0.896962,...,-0.734879,-0.248222,-0.925588,-0.629434,0.034498,0.470001,-0.825195,-0.385860,-0.915567,-0.934467
7,-0.331746,-0.42059,0.490433,-1.228373,-0.157977,0.132171,-0.003928,0.090947,0.461727,-0.157977,...,0.637256,0.544202,-0.161121,1.660838,0.657876,-0.785154,0.423769,0.049912,-0.314831,0.716430
13,-0.765662,-0.42059,0.490433,-0.122048,-1.002531,2.305399,0.291300,0.478319,0.547425,-1.002531,...,-1.954555,-0.823947,-0.999808,-1.785766,0.261547,0.156213,-0.921269,-0.680613,-0.915567,-0.974584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206202,-0.223266,-0.42059,-0.857505,-0.660011,-0.221319,-0.688576,-2.070525,1.253065,0.823048,-0.221319,...,-0.277501,-0.025720,-0.183387,-0.555324,0.553565,-0.471365,-0.344824,-0.226590,-0.164647,-0.058848
206206,2.217516,-0.42059,0.490433,-0.540308,0.510627,0.241980,1.472213,0.090947,1.851516,0.510627,...,-1.649636,-1.554117,0.603346,-1.365685,-0.423468,-0.785154,0.423769,-1.032167,2.013021,-0.483118
206207,-0.548704,-0.42059,0.490433,-0.181198,-0.129826,0.505864,-0.889612,0.478319,0.290264,-0.129826,...,0.637256,0.810925,-0.220497,2.048019,-0.087674,0.470001,0.519843,0.667692,-0.615199,0.659860


In [ ]:
# Export
user_feat_comb.to_parquet("../data/feature_agg/train_user_features.parquet")
np.save("../data/feature_agg/train_user_features_norm.npy", user_feat_comb_norm.values)

In [65]:
data_list = [data, data]
loader = DataLoader(data_list, batch_size=2)
batch = next(iter(loader))

In [66]:
batch

BipartiteDataBatch(edge_index=[2, 1322104], x_s=[99378, 768], x_t=[99378, 768], batch=[412420], ptr=[3])